# Max Healthcare AI/ML Assignment

This notebook implements a robust CNN-based classifier for dermatology images with noisy training labels and clean validation labels.

## Handling Noisy Labels

Medical image datasets often suffer from annotation noise due to inter-observer variability. Since training labels are noisy while validation labels are expert-verified, the goal is to build a model that generalizes well rather than memorizing noisy samples.

To address this, label smoothing and architectural regularization (dropout, batch normalization, global average pooling) are used to improve robustness.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

from model_definition import DermCNN

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# Load dataset (path will be replaced during live test)
data = np.load('candidate_dataset.npz')
X = torch.tensor(data['images']).float().unsqueeze(1) / 255.0
y = torch.tensor(data['labels']).long()

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# Model, loss, optimizer
model = DermCNN(num_classes=7)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
train_losses = []

for epoch in range(5):
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

In [ ]:
# Plot training loss
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()

In [ ]:
# Save trained model
import os
os.makedirs('../model', exist_ok=True)
torch.save(model.state_dict(), '../model/best_dermcnn.pth')
print('Model saved to model/best_dermcnn.pth')